In [ ]:
import torch
import torch.nn as nn
from torch.optim import lr_scheduler
from torch.autograd import Variable
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import torch.nn.functional as F
import matplotlib.pyplot as plt
import glob
import os
from skimage.transform import resize
import cv2
import itertools
import warnings
warnings.filterwarnings('ignore')
import skimage as sk
sk.__version__

In [ ]:
"""pair_training_loss = 0.0
            pair_training_corrects = 0.0
            pairs = list(pairs)

            label = 0
            k = 0

            for folder in pairs:
                if folder is not None:            
                    X,Y = create_sets(folder,label)
                    if k == 0:
                        X_train_batch = X.copy()
                        Y_train_batch = Y.copy()
                    else:
                        X_train_batch = np.concatenate([X_train_batch,X],axis=0)
                        Y_train_batch = np.concatenate([Y_train_batch,Y],axis=0)
                    k += 1
                    
                label += 1
            

            print(X_train_batch.shape,Y_train_batch.shape)
            num_samples = Y_train_batch.shape[0]
            print("num_samples: "+str(num_samples))
            X_train_batch = normalize(X_train_batch)
            sub_batches = create_batches(X_train_batch,Y_train_batch,batch_size)
            for X_minibatch,Y_minibatch in sub_batches:
                Y_minibatch = one_hot(Y_minibatch,num_classes)
                X_minibatch, Y_minibatch = torch.from_numpy(X_minibatch.reshape(X_minibatch.shape[0],3,224,224)).to(device).float(), torch.from_numpy(Y_minibatch).to(device).float()
                output = compute_output(X_minibatch,model)
                loss,correct = evaluate(output,Y_minibatch)
                optimizer.zero_grad()
                loss.backward()
                optimizer.step()
                pair_training_loss += loss.item()
                pair_training_corrects += correct"""

In [ ]:
"""res = models.resnet34(pretrained=True)
for param in res.parameters():
        param.requires_grad = False
p = 0
for child in res.children():
    print(child)
    print(p)
    if p >= 5:
        for param in child.parameters():
            param.requires_grad = True
            
    p+=1
    
    
class CNN(nn.Module):
    def __init__(self, original_model):
        super(CNN, self).__init__()
        self.feature_extractor = *list(original_model.children())[:-1]
        self.classifier = nn.Linear(in_features=512,out_features=99,bias=True)
    def forward(x):
        x = self.feature_extractor(x)
        x = self.classifier(x)
        x = F.softmax(x,dim=1)
        return x

cnn = CNN(res)
#cnn"""

In [ ]:
def create_pair_lists(train_test_path):
    
    class_names = [names for names in glob.glob(train_test_path+"/*")]
    #print(class_names)
    
    a = []
    for names in class_names:
        folder = []
        for folders in glob.glob(names+"/*"):
            folder.append(folders)
        a.append(folder)
        

    paired_folders = itertools.zip_longest(*a)
    
    return list(paired_folders)

pair_training = create_pair_lists(train_test_path= "Frames/Train")
pair_test =  create_pair_lists(train_test_path= "Frames/Test")
#print(list(pair[0]))

In [ ]:
def im_toarray(im):
    array = plt.imread(im)
    array = resize(array,(224,224,3),anti_aliasing=True)
    return array

    


In [ ]:
def create_sets(folder,label):
    X_ = []
    Y_ = []
    frame_list = []
    for frame in glob.glob(folder+"/*.jpg"):
        im_array = im_toarray(frame)
        X_.append(im_array)
        Y_.append(label)
        frame_list.append(frame)
    
    return np.array(X_),np.array(Y_).reshape(-1,1),frame_list
        

In [ ]:
def one_hot(array, num_classes):
    return np.squeeze(np.eye(num_classes)[array.reshape(-1)])

In [ ]:
def normalize(X):
    mean = np.array([0.485, 0.456, 0.406])
    std = np.array([0.229, 0.224, 0.225])
    
    X_normalized = (X - mean)/std
    return X_normalized
    

In [ ]:
def model():

    model = models.resnet18(pretrained=True)

    model.fc = nn.Linear(in_features=512, out_features=99, bias=True)
    
    for param in model.parameters():
        param.requires_grad = False

    p = 0
    for child in model.children():
        #print(p)
        if p >=3:
            for param in child.parameters():
                param.requires_grad = True
        p += 1
    
    return model

class RESNET(nn.Module):
    def __init__(self, original_model):
        super(RESNET, self).__init__()
        self.feature_extractor = nn.Sequential(*list(original_model.children())[:-1])
        self.classifier = nn.Sequential(*list(original_model.children())[-1:])
    def forward(self,x):
        (m,nc,nw,nh) = x.size()
        x = self.feature_extractor(x).view(m,-1)
        x = self.classifier(x)
        x = F.softmax(x,dim=1)
        return x
    
"""def compute_output(X,model):
    out = F.softmax(model.forward(X),dim=1)
    return out"""

device = torch.device("cuda:0")
model = model()
#model_parameters = filter(lambda p: p.requires_grad, model.parameters())
#params = sum([np.prod(p.size()) for p in model_parameters])       
res = RESNET(model).to(device).float()
#a = torch.randn(1,3,224,224).cuda().float()
#res(a)

In [ ]:
def create_batches(X,Y,all_frame_list,batch_size,shuffle=True):
    l = Y.shape[0]
    index = np.random.permutation(l)
    X = X[index,:,:,:]
    Y = Y[index,:]
    all_frame_list = np.array(all_frame_list)
    all_frame_list = all_frame_list[index]
    #print(len(Y))
    for i in range(0,Y.shape[0],batch_size):
        yield(X[i:i+batch_size,:,:,:],Y[i:i+batch_size,:],all_frame_list[i:i+batch_size])
    
    

    

In [ ]:
def save_features(frame_array,conv_features):
    #print(conv_features.detach().cpu().numpy().shape)
    #print(frame_array)
    for k in range(len(frame_array)):
        jpg_path = frame_array[k]
        #print(jpg_path)
        npy_path = jpg_path[:-3] + "npy"
        #print(npy_path)
        array = conv_features[k,:,:,:]
        np.save(npy_path,array.numpy())
    

In [ ]:
def train_stuff(train_or_test,pairs,batch_size,num_classes,optimizer,model):

        pair_loss = 0.0
        pair_corrects = 0.0
        pairs = list(pairs)
        #print(pairs)
        label = 0
        k = 0
        
        all_frame_list = []
        for folder in pairs:
            if folder is not None:            
                X,Y,frame_list = create_sets(folder,label)
                all_frame_list.append(frame_list)
                if k == 0:
                    X_batch = X.copy()
                    Y_batch = Y.copy()
                else:
                    X_batch = np.concatenate([X_batch,X],axis=0)
                    Y_batch = np.concatenate([Y_batch,Y],axis=0)
                k += 1
                    
            label += 1
        all_frame_list  = [val for sublist in all_frame_list for val in sublist] 
        #print(len(all_frame_list))
        print(X_batch.shape,Y_batch.shape)
        num_samples = Y_batch.shape[0]
        print("num_samples: "+str(num_samples))
        X_batch = normalize(X_batch)
        sub_batches = create_batches(X_batch,Y_batch,all_frame_list,batch_size)
        for X_minibatch,Y_minibatch,frame_minibatch in sub_batches:
            
            Y_minibatch = one_hot(Y_minibatch,num_classes)
            X_minibatch, Y_minibatch = torch.from_numpy(X_minibatch.reshape(X_minibatch.shape[0],3,224,224)).to(device).float(), torch.from_numpy(Y_minibatch.reshape(-1,num_classes)).to(device).float()
            conv_features = model.feature_extractor(X_minibatch)
            #classifier = model.classifier(conv_features)
            output = model(X_minibatch)
            
            loss,correct = evaluate(output,Y_minibatch)
            if train_or_test == "train":
                optimizer.zero_grad()
                loss.backward()
                optimizer.step()
            pair_loss += loss.item()
            pair_corrects += correct
            save_features(frame_minibatch,conv_features)
        
        return pair_loss,pair_corrects,num_samples,model
        

In [ ]:
def train_model(model,nb_epoch,learning_rate,paired_training_list,paired_test_list,device,batch_size):
    
    optims = filter(lambda p: p.requires_grad,model.parameters())    
    optimizer = torch.optim.Adam(optims,lr=learning_rate)
    print("There are total number of %d pairs in training set"%len(paired_training_list))
    print("There are total number of %d pairs in test set"%len(paired_test_list))
    num_classes = 99
    
    for epoch in range(1,nb_epoch+1):

        train_epoch_loss = 0.0
        test_epoch_loss = 0.0
        pair_nu = 1
        for train_pairs in paired_training_list:
            
            pair_training_loss,pair_training_corrects,num_samples,model = train_stuff("train",train_pairs,batch_size,num_classes,optimizer,model)
            print("pair: "+str(pair_nu))
            print("pair training loss:" +str(pair_training_loss))
            print("pair training accuracy: "+str(100*pair_training_corrects/num_samples))
            pair_nu += 1
            """pair_test_loss,pair_test_corrects,num_samples,model = train_stuff("test",test_pairs,batch_size,num_classes,optimizer,model)    
            print("pair test loss:" +str(pair_test_loss))
            print("pair test accuracy: "+str(100*pair_test_corrects/num_samples))
            
            train_epoch_loss += pair_training_loss
            test_epoch_loss += pair_test_loss"""
            
        
            


        print("epoch: "+str(epoch))
        print("epoch loss: " +str(epoch_loss/num_classes))
    
        
            
        

In [ ]:
def evaluate(out,real):
    #print(out.size(),real.size())
    log_loss = torch.mean(-real*torch.log(out))
    mean_loss = torch.mean((out-real)**2)
    
    loss = (mean_loss + log_loss)/2.0
    
    real_arg = torch.argmax(real,dim=1)
    out_arg = torch.argmax(out,dim=1)
    #print(sum(out_arg==real_arg).item())
    correct = sum(out_arg==real_arg).item()
    return log_loss,correct
        
    
    

In [ ]:
train_model(res,2,1.e-3,pair_training,pair_test,device,32)

In [ ]:
a = np.array([0,1,1,2,3,3,3,4,5,5,5,5])
b = np.array([1,0,0,2,4,3,5,5,4,4,5,1])

a = torch.from_numpy(one_hot(a,7)).float()
b = torch.from_numpy(one_hot(b,7)).float()

loss,accuracy = evaluate(a,b)
print(loss,accuracy)

In [ ]:
"""def call_data(selected_videos,class_path,label):
    X, Y = [], []
    #class_names = [names for names in glob.glob(path+"/*")]
    
    video_folders = [video for video in glob.glob(class_path+"/*")]
    #print(video_folders)
    shuffled_videos = list(np.random.permutation(video_folders))
    unused_videos = list(set(shuffled_videos) - set(selected_videos))
    if len(unused_videos) != 0:
        selected_video = unused_videos[0]
        selected_videos.append(selected_video)
        
        for frame in glob.glob(selected_video+"/*.jpg"):
            im_array = im_toarray(frame)
            X.append(im_array)
            Y.append(label)
    
    return np.array(X),np.array(Y),selected_videos"""

            

In [ ]:
b = np.array([1,1,1,1,1,2,2,4,4,9,9,9])
a = np.array([1,1,1,1,1,2,2,4,4,9,9,9])

on = one_hot(b,10)
#b_n = normalize(b)
#b_n
on

In [ ]:
on_t = torch.from_numpy(on)
print(torch.argmax(on_t,dim=1).topk(5))
arg = np.argmax(on,axis=1)
#on_t = torch.from_numpy(on)
#argt = torch.argmax(on_t,dim=1)
#arg,arg_t


In [ ]:
i = [1,2,3,4,5]

k = [["asd/frame1.jpg"],["fgh/frame0.jpg"]]
j = np.random.permutation(4)
k = np.array(k)
jpg = k[0][0]
jpg[:-3] + "npy"
i[-2:]